In [1]:
import os
import shutil
import yaml
from glob import glob
from ultralytics import YOLO
from roboflow import Roboflow

# --- 1. 기본 경로 및 API 키 설정 ---
# 기존 데이터셋과 최종 결과가 저장될 상위 폴더 경로
BASE_DATA_PATH = 'C:/Users/Admin/work space/2nd/'

# Roboflow API 키를 입력해주세요.
ROBOFLOW_API_KEY = "NjIXpou4o4gsuGClT8hI" # ⬅️ YOUR API KEY HERE

# 기존 데이터셋 폴더 이름
DATASET1_NAME = 'acne04-1'
DATASET2_NAME = 'Acne-detection-1'

# 새로 다운로드할 데이터셋 정보
ROBOFLOW_WORKSPACE = "test-l2t0m"
ROBOFLOW_PROJECT = "acne-detection-hjgkp-cvltk"
ROBOFLOW_VERSION = 1
# Roboflow 다운로드 시 생성될 폴더 이름 (프로젝트 이름-버전)
DATASET3_NAME = f"{ROBOFLOW_PROJECT}-{ROBOFLOW_VERSION}"

# 최종 통합 데이터셋이 저장될 폴더 이름
COMBINED_DATASET_NAME = 'acne_dataset_final_retrained'

# 이전에 학습시킨 모델의 best.pt 파일 경로
# 이 모델을 불러와 추가 학습(Fine-tuning)을 진행합니다.
PREVIOUS_MODEL_PATH = os.path.join(BASE_DATA_PATH, 'acne_model_8_classes', 'weights', 'best.pt')



In [2]:
# --- 2. 새로운 Roboflow 데이터셋 다운로드 ---
print("🚀 Roboflow 데이터셋 다운로드를 시작합니다...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(ROBOFLOW_VERSION)
# 다운로드 경로를 BASE_DATA_PATH로 지정
dataset = version.download("yolov11", location=BASE_DATA_PATH)
print("✅ Roboflow 데이터셋 다운로드 완료!")
print("-" * 50)

🚀 Roboflow 데이터셋 다운로드를 시작합니다...
loading Roboflow workspace...
loading Roboflow project...
✅ Roboflow 데이터셋 다운로드 완료!
--------------------------------------------------


In [3]:
# --- 3. 클래스 목록 정의 및 중복 클래스 확인 ---
# 기존에 사용했던 최종 클래스 목록
FINAL_CLASS_NAMES = sorted(['blackheads', 'comedone', 'cyst', 'fore', 'nodule', 'papule', 'pustule', 'whiteheads'])

# 새로 다운로드한 데이터셋의 data.yaml 파일에서 클래스 이름 로드
new_dataset_yaml_path = os.path.join(dataset.location, 'data.yaml')
new_d3_names = []
if os.path.exists(new_dataset_yaml_path):
    with open(new_dataset_yaml_path, 'r') as f:
        new_yaml_data = yaml.safe_load(f)
        new_d3_names = new_yaml_data.get('names', [])

# 기존 클래스와 새로운 클래스 간의 중복(교집합) 찾기
final_class_set = set(FINAL_CLASS_NAMES)
new_d3_names_set = set(new_d3_names)
overlapping_classes = sorted(list(final_class_set.intersection(new_d3_names_set)))

print(f"✅ 기존 최종 클래스 목록 (총 {len(FINAL_CLASS_NAMES)}개):")
print(FINAL_CLASS_NAMES)
print(f"\n✅ 새로 추가할 데이터셋의 클래스 (총 {len(new_d3_names)}개):")
print(new_d3_names)
print(f"\n✅ 중복되어 추가 학습에 사용될 클래스 (총 {len(overlapping_classes)}개):")
print(overlapping_classes)
print("-" * 50)

✅ 기존 최종 클래스 목록 (총 8개):
['blackheads', 'comedone', 'cyst', 'fore', 'nodule', 'papule', 'pustule', 'whiteheads']

✅ 새로 추가할 데이터셋의 클래스 (총 0개):
[]

✅ 중복되어 추가 학습에 사용될 클래스 (총 0개):
[]
--------------------------------------------------


In [4]:
# --- 4. 클래스 ID 재매핑 테이블 생성 ---
# 최종 클래스 이름을 기준으로 새로운 ID 부여 (name -> id)
final_name_to_id = {name: i for i, name in enumerate(FINAL_CLASS_NAMES)}

# 각 데이터셋의 원래 ID를 -> 최종 ID로 변환하는 테이블 생성
# 데이터셋 1, 2는 기존 방식과 동일
ORIGINAL_D1_NAMES = ['fore', 'papule', 'pustule', 'nodule', 'whiteheads', 'cyst', 'blackheads']
original_d1_name_to_id = {name: i for i, name in enumerate(ORIGINAL_D1_NAMES)}
remap_d1 = {orig_id: final_name_to_id[name] for name, orig_id in original_d1_name_to_id.items()}

ORIGINAL_D2_NAMES = ['comedone', 'pustule', 'papule', 'cyst', 'nodule']
original_d2_name_to_id = {name: i for i, name in enumerate(ORIGINAL_D2_NAMES)}
remap_d2 = {orig_id: final_name_to_id[name] for name, orig_id in original_d2_name_to_id.items()}

# 데이터셋 3 (신규) 매핑 테이블: 중복 클래스에 대해서만 생성
original_d3_name_to_id = {name: i for i, name in enumerate(new_d3_names)}
remap_d3 = {
    orig_id: final_name_to_id[name]
    for name, orig_id in original_d3_name_to_id.items()
    if name in overlapping_classes
}
original_d3_id_to_name = {i: name for name, i in original_d3_name_to_id.items()}



In [5]:
# --- 5. 데이터 병합 및 라벨 재매핑 실행 ---
src_path_d1 = os.path.join(BASE_DATA_PATH, DATASET1_NAME)
src_path_d2 = os.path.join(BASE_DATA_PATH, DATASET2_NAME)
src_path_d3 = dataset.location # Roboflow 다운로드 경로
combined_path = os.path.join(BASE_DATA_PATH, COMBINED_DATASET_NAME)

# 기존에 폴더가 있다면 삭제하고 새로 시작
if os.path.exists(combined_path):
    shutil.rmtree(combined_path)
    print(f"기존 '{combined_path}' 폴더를 삭제했습니다.")

print(f"'{combined_path}' 폴더를 생성하고 3개 데이터셋 병합을 시작합니다...")

# 데이터셋을 처리하는 함수
def process_dataset(src_path, remap_dict, dest_path, classes_to_keep=None, id_to_name_map=None):
    for split in ['train', 'valid', 'test']:
        src_label_dir = os.path.join(src_path, split, 'labels')
        src_image_dir = os.path.join(src_path, split, 'images')

        if not os.path.exists(src_image_dir):
            continue

        dest_label_dir = os.path.join(dest_path, split, 'labels')
        dest_image_dir = os.path.join(dest_path, split, 'images')

        os.makedirs(dest_label_dir, exist_ok=True)
        os.makedirs(dest_image_dir, exist_ok=True)

        # 라벨 파일 처리 및 이미지 복사
        for label_filename in glob(os.path.join(src_label_dir, '*.txt')):
            base_filename = os.path.basename(label_filename)
            
            with open(label_filename, 'r') as f_in:
                lines = f_in.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if not parts: continue
                
                original_id = int(parts[0])
                
                # 클래스 필터링 (새로운 데이터셋에만 적용)
                if classes_to_keep and id_to_name_map:
                    class_name = id_to_name_map.get(original_id)
                    if class_name not in classes_to_keep:
                        continue # 중복 클래스가 아니면 건너뛰기

                if original_id in remap_dict:
                    parts[0] = str(remap_dict[original_id])
                    new_lines.append(' '.join(parts) + '\n')
            
            # 유효한 라벨이 있는 경우에만 파일 생성 및 이미지 복사
            if new_lines:
                with open(os.path.join(dest_label_dir, base_filename), 'w') as f_out:
                    f_out.writelines(new_lines)

                # 해당 이미지 파일 복사 (다양한 확장자 고려)
                img_name_base = os.path.splitext(base_filename)[0]
                for ext in ['.jpg', '.jpeg', '.png']:
                    img_filename = img_name_base + ext
                    src_image_path = os.path.join(src_image_dir, img_filename)
                    if os.path.exists(src_image_path):
                        shutil.copy(src_image_path, os.path.join(dest_image_dir, img_filename))
                        break


# 각 데이터셋 처리 실행
print("1/3 - 기존 데이터셋 1 처리 중...")
process_dataset(src_path_d1, remap_d1, combined_path)
print("2/3 - 기존 데이터셋 2 처리 중...")
process_dataset(src_path_d2, remap_d2, combined_path)
print("3/3 - 새로운 데이터셋 (중복 클래스만) 처리 중...")
process_dataset(src_path_d3, remap_d3, combined_path, classes_to_keep=overlapping_classes, id_to_name_map=original_d3_id_to_name)


print("✅ 데이터 병합 및 라벨 재매핑 완료!")
print("-" * 50)

기존 'C:/Users/Admin/work space/2nd/acne_dataset_final_retrained' 폴더를 삭제했습니다.
'C:/Users/Admin/work space/2nd/acne_dataset_final_retrained' 폴더를 생성하고 3개 데이터셋 병합을 시작합니다...
1/3 - 기존 데이터셋 1 처리 중...
2/3 - 기존 데이터셋 2 처리 중...
3/3 - 새로운 데이터셋 (중복 클래스만) 처리 중...
✅ 데이터 병합 및 라벨 재매핑 완료!
--------------------------------------------------


In [6]:
# --- 6. 최종 data.yaml 파일 생성 ---
final_yaml_path = os.path.join(combined_path, 'data.yaml')

yaml_data = {
    'train': os.path.join('train', 'images'),
    'val': os.path.join('valid', 'images'),
    'test': os.path.join('test', 'images'),
    'nc': len(FINAL_CLASS_NAMES),
    'names': FINAL_CLASS_NAMES
}

with open(final_yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(yaml_data, f, allow_unicode=True, sort_keys=False)

print(f"✅ '{final_yaml_path}' 파일 생성 완료!")
print("--- data.yaml 내용 ---")
print(yaml.dump(yaml_data, allow_unicode=True, sort_keys=False))
print("-" * 50)

✅ 'C:/Users/Admin/work space/2nd/acne_dataset_final_retrained\data.yaml' 파일 생성 완료!
--- data.yaml 내용 ---
train: train\images
val: valid\images
test: test\images
nc: 8
names:
- blackheads
- comedone
- cyst
- fore
- nodule
- papule
- pustule
- whiteheads

--------------------------------------------------


In [7]:
# --- 7. 모델 추가 학습 (Fine-tuning) ---
if not os.path.exists(PREVIOUS_MODEL_PATH):
     print(f"🚨 에러: 이전에 학습된 모델 '{PREVIOUS_MODEL_PATH}'를 찾을 수 없습니다.")
     print("Fine-tuning을 진행할 수 없습니다. 경로를 확인해주세요.")
else:
    print(f"🚀 기존 모델 '{PREVIOUS_MODEL_PATH}'를 불러와 추가 학습(Fine-tuning)을 시작합니다...")
    
    # 이전에 학습된 모델 로드
    model = YOLO(PREVIOUS_MODEL_PATH)
    
    # 모델 학습 실행 (workers 항목 추가)
    results = model.train(
        data=final_yaml_path,
        epochs=100,      # 추가 학습이므로 에포크 수를 줄여서 설정 (필요시 조정)
        imgsz=640,
        batch=16,
        patience=50,   # 성능 개선이 없을 때 조기 종료하는 기준 (에포크)
        project=BASE_DATA_PATH,
        name='acne_model_8_classes_retrained', # 새로운 결과 폴더 이름
        workers=2        # ⬅️ 이 항목을 추가했습니다! (메모리가 부족하면 2로 줄여보세요)
    )
    
    print("\n🎉 모든 과정이 완료되었습니다! 학습 결과는 다음 폴더에 저장되었습니다:")
    print(os.path.join(BASE_DATA_PATH, 'acne_model_8_classes_retrained'))

🚀 기존 모델 'C:/Users/Admin/work space/2nd/acne_model_8_classes\weights\best.pt'를 불러와 추가 학습(Fine-tuning)을 시작합니다...
New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/Admin/work space/2nd/acne_dataset_final_retrained\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr

In [8]:
 # --- 8. 최종 모델 성능 검증 ---
print("\n🚀 테스트 데이터셋으로 최종 모델 성능을 검증합니다...")
    
    # 가장 성능이 좋았던 best.pt 모델 로드
best_model_path = os.path.join(BASE_DATA_PATH, 'acne_model_8_classes_retrained', 'weights', 'best.pt')
model = YOLO(best_model_path)

metrics = model.val(data=final_yaml_path, split='test', imgsz=640)
    
print("\n--- 최종 성능 지표 ---")
print(f"mAP50-95 (평균 IoU 0.5~0.95): {metrics.box.map:.4f}")
print(f"mAP50 (IoU 0.5 기준): {metrics.box.map50:.4f}")
print(f"mAP75 (IoU 0.75 기준): {metrics.box.map75:.4f}")


🚀 테스트 데이터셋으로 최종 모델 성능을 검증합니다...
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 2.91.3 MB/s, size: 16.3 KB)
val: Scanning C:\Users\Admin\work space\2nd\acne_dataset_final_retrained\test\labels... 88 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 88/88 466.9it/s 0.2s0.3s
val: New cache created: C:\Users\Admin\work space\2nd\acne_dataset_final_retrained\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 0.8it/s 8.0s0.4ss
                   all         88        461      0.579      0.434      0.453      0.185
              comedone         24         86      0.754      0.499      0.632       0.22
                  cyst         18         19      0.776      0.789       0.76      0.359
                nodule         19      